<h1>Land Feature CNN</h1>

Code and network framework gathered from:
https://www.tensorflow.org/tutorials/images/cnn

Authors:
- Thao Pham
- Lawrence Hoerst


In [ ]:
import theCropper
theCropper.main()

In [ ]:
import theScaler
theScaler.main()

In [ ]:
import converter
# This will load the colored images into np arrays and pickle them
converter.main()

In [ ]:
# Import tensorflow for the model
import tensorflow as tf
from tensorflow.keras import layers, models
# Import matplotlib to show retults
import matplotlib.pyplot as plt
# Import numpy for data processing
import numpy as np
# Import from PIL
from PIL import Image
from PIL import ImageColor
# Import pickle to read in training data
import pickle
# Import os to work with filepaths
import os
# Import time for sleep
import time

In [ ]:
# Needs to be reordered to match the new order of the classes
# # Class names to plot the images:
# class_names = ['Water', 'Buildings', 'Roads', 'Foliage', 'Mineral deposits', 'Mountainous terrain', 'Rocky terrain', 'Sandy terrain', 'Plains', 'Snow', 'Grass']
# class_colors = ['#0f5e9c', ('#f2f2f2', '#606060'), '#c4c4c4', '#3a5f0b', '#490e0e', '#5a7a4c', '#698287', '#f7ae64', '#c89e23', '#fffafa', '#7cfc00']
# class_codes = {class_names[i]: i for i in range(len(class_names))}

# # hex_to_rgb
# cvt = lambda hex: ImageColor.getcolor(hex, "RGB")
# colors_to_code = {cvt('#0f5e9c'): 0,
#                   cvt('#f2f2f2'): 1, cvt('#606060'): 1,
#                   cvt('#c4c4c4'): 2,
#                   cvt('#3a5f0b'): 3,
#                   cvt('#490e0e'): 4,
#                   cvt('#5a7a4c'): 5,
#                   cvt('#698287'): 6,
#                   cvt('#f7ae64'): 7,
#                   cvt('#c89e23'): 8,
#                   cvt('#fffafa'): 9,
#                   cvt('#7cfc00'): 10,
#                   'default':      11}

In [ ]:
# Instantiate the model
model = models.Sequential()
# Add layers
# We need to decide on how many layers we want
# These parameters are not yet setup for our network
model.add(layers.InputLayer(shape=(*theScaler.scalingDimensions, 4)))
model.add(layers.Conv2D(32, (3, 3), activation='sigmoid'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
model.add(layers.Flatten())
model.add(layers.Dense(theScaler.scalingDimensions[0] * theScaler.scalingDimensions[1], activation='sigmoid'))
model.add(layers.Dense(theScaler.scalingDimensions[0] * theScaler.scalingDimensions[1], activation='sigmoid'))
model.add(layers.Reshape((*theScaler.scalingDimensions, 1))) # Make output in the same shape as a classification image
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError(name="accuracy")])

In [ ]:
print("Shape of input: ",(*theScaler.scalingDimensions, 4))
print(model.summary())

In [ ]:
# unpickle our test data here
input_sizing = lambda array: [np.reshape(value, (-1, *theScaler.scalingDimensions, 4)) for value in array]
output_sizing = lambda array: [np.reshape(value, (-1, *theScaler.scalingDimensions, 1)) for value in array]
resize = False
with open('training_inputs.pickle', 'rb') as inputsFile:
    training_inputs = pickle.load(inputsFile)
    if resize: training_inputs = input_sizing(training_inputs)
with open('training_outputs.pickle', 'rb') as outputsFile:
    training_outputs = pickle.load(outputsFile)
    if resize: training_outputs = output_sizing(training_outputs)

# unpickle the validation / testing outputs
with open('validation_inputs.pickle', 'rb') as inputsFile:
    validation_inputs = pickle.load(inputsFile)
    if resize: validation_inputs = input_sizing(validation_inputs)
with open('validation_outputs.pickle', 'rb') as outputsFile:
    validation_outputs = pickle.load(outputsFile)
    if resize: validation_outputs = output_sizing(validation_outputs)

print_data = lambda func: print(f"training inputs: {func(training_inputs)}, training outputs: {func(training_outputs)}, validation inputs: {func(validation_inputs)}, and validation outputs: {func(validation_outputs)}")
print_data(len)
print_data(lambda e: e[0].shape)



In [ ]:
# Train the model
history = model.fit(training_inputs, training_outputs, epochs=10, validation_data=(validation_inputs, validation_outputs))

In [ ]:

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(validation_inputs,  validation_outputs, verbose=2)


In [ ]:
def predict(np_array):
    prediction = model.predict(np.reshape(np_array, (-1, *theScaler.scalingDimensions, 4)))
    prediction = np.reshape(prediction, (*theScaler.scalingDimensions, 1))
    prediction_decoded = converter.decode_image(prediction)
    predicted_image = Image.fromarray(prediction_decoded)
    predicted_image.show()
    print(prediction)
def displayImage(np_array):
    image = Image.fromarray(np_array)
    image.show()
# displayImage(training_inputs[0])
# predict(training_inputs[0])
# time.sleep(5)
# displayImage(training_inputs[1])
# predict(training_inputs[1])
# time.sleep(5)
# displayImage(training_inputs[2])
# predict(training_inputs[2])
# time.sleep(5)
print(training_outputs[42])